## 第9章 建立基于特征的文法

自然语言具有范围广泛的**文法结构**，用第8章中所描述的简单的方法很难处理的如此广泛的文法结构。为了获得更大的灵活性，我们改变我们对待文法类别如 `S`、`NP` 和 `V` 的方式。 我们将这些原子标签分解为类似字典的结构，以便可以提取一系列的值作为特征。 

本章的目的是要回答下列问题： 
1. 我们怎样用特征扩展上下文无关文法框架，以获得更细粒度的对文法类别和产生式的控制？
2. 特征结构的主要形式化属性是什么，我们如何使用它们来计算？ 
3. 我们现在用基于特征的文法能捕捉到什么语言模式和文法结构？ 

一路上，我们将介绍更多的英语句法主题，包括：约定、子类别和无限制依赖成分等现象。

### 9.1 文法特征

在第6 章，我们描述了如何通过检测文本的特征建立分类器。那些特征可能非常简单，如提取一个单词的最后一个字母，或者更复杂一点儿，如分类器自己预测的词性标签。在本章中，我们将探讨在建立基于规则的文法中特征的作用。对比特征提取，记录已经自动检测到的特征，我们现在要声明词和短语的特征。我们以一个很简单的例子开始，使用字典存储特征和它们的值。

In [1]:
kim = {'CAT': 'NP', 'ORTH': 'Kim', 'REF': 'k'}
chase = {'CAT': 'V', 'ORTH': 'chased', 'REL': 'chase'}

对象`kim` 和`chase` 有几个共同的特征，**`CAT`（文法类别）**和 **`ORTH`（正字法，即拼写）**。
此外，每一个还有更面向语义的特征：`kim['REF']`意在给出 `kim` 的指示物，而 `chase['REL']`给出 `chase` 表示的关系。在基于规则的文法上下文中，这样的特征和特征值对被称为**特征结构**，我们将很快看到它们的替代符号。

特征结构包含各种有关文法实体的信息。这些信息不需要详尽无遗，我们可能要进一步增加属性。例如：一个动词，知道它扮演的“语义角色”往往很有用。对于`chase`，主语扮演“`agent`（施事）”的角色，而宾语扮演“`patient`（受事）”角色。让我们添加这些信息，使 用`'sbj'`（主语）和`'obj'`（宾语）作为占位符，它会被填充，当动词和它的文法参数结合时：

In [2]:
chase['AGT'] = 'sbj'
chase['PAT'] = 'obj'

如果我们现在处理句子：`Kim chased Lee`，我们要“绑定”动词的**施事角色**和**主语**，**受事角色和宾语**。我们可以通过链接到相关的 `NP` 的 `REF` 特征做到这个。在下面的例子中， 我们做一个简单的假设：在动词直接左侧和右侧的 `NP` 分别是主语和宾语。我们还在例子结尾为 `Lee` 添加了一个特征结构。

In [7]:
sent = "Kim chased Lee"
tokens = sent.split()
lee = {'CAT': 'NP', 'ORTH': 'Lee', 'REF': 'l'}
def lex2fs(word):
    for fs in [kim, lee, chase]:
        if fs['ORTH'] == word:
            return fs
subj, verb, obj = lex2fs(tokens[0]), lex2fs(tokens[1]), lex2fs(tokens[2])
verb['AGT'] = subj['REF'] # # agent of 'chase' is Kim
verb['PAT'] = obj['REF'] # patient of 'chase' is Lee
for k in ['ORTH', 'REL', 'AGT', 'PAT']:
    print("%-5s => %s" % (k, verb[k]))

ORTH  => chased
REL   => chase
AGT   => k
PAT   => l


特征结构是非常强大的，但我们操纵它们的方式是极其特别的。我们本章接下来的任务是，显示上下文无关文法和分析如何能扩展到合适的特征结构，使我们可以一种更通用的和有原则的方式建立像这样的分析。我们将通过查看句法协议的现象作为开始；我们将展示如何使用特征典雅的表示协议约束，并在一个简单的文法中说明它们的用法。

由于**特征结构是表示任何形式的信息的通用的数据结构**，我们将从更形式化的视点简要地看着它们，并演示 NLTK 提供的特征结构的支持。在本章的最后一部分，我们将表明，特征的额外表现力开辟了一个用于描述语言结构的复杂性的广泛的可能性。


#### 句法协议

在英语中，名词通常被标记为单数或复数。要么两个都是单数要么都是复数。主语和谓词间也存在类似的约束。这里我们可以看到，动词的形态属性与主语名词短语的句法属性一起变化。这种一起变化被称为**协议**（`agreement`）。如果我们进一步看**英语动词协议**，我们将看到动词的现在时态通常有两种屈折形式：一为**第三人称单数**，另一个为**人称和数量的所有其他组合**。动词与它的主语在人称和数量上保持一致。

#### 使用属性和约束

我们说过非正式的语言类别具有属性，例如：名词具有复数的属性。

In [8]:
# 例9-1. 基于特征的文法的例子
import nltk
nltk.data.show_cfg('grammars/book_grammars/feat0.fcfg')

% start S
# ###################
# Grammar Productions
# ###################
# S expansion productions
S -> NP[NUM=?n] VP[NUM=?n]
# NP expansion productions
NP[NUM=?n] -> N[NUM=?n] 
NP[NUM=?n] -> PropN[NUM=?n] 
NP[NUM=?n] -> Det[NUM=?n] N[NUM=?n]
NP[NUM=pl] -> N[NUM=pl] 
# VP expansion productions
VP[TENSE=?t, NUM=?n] -> IV[TENSE=?t, NUM=?n]
VP[TENSE=?t, NUM=?n] -> TV[TENSE=?t, NUM=?n] NP
# ###################
# Lexical Productions
# ###################
Det[NUM=sg] -> 'this' | 'every'
Det[NUM=pl] -> 'these' | 'all'
Det -> 'the' | 'some' | 'several'
PropN[NUM=sg]-> 'Kim' | 'Jody'
N[NUM=sg] -> 'dog' | 'girl' | 'car' | 'child'
N[NUM=pl] -> 'dogs' | 'girls' | 'cars' | 'children' 
IV[TENSE=pres,  NUM=sg] -> 'disappears' | 'walks'
TV[TENSE=pres, NUM=sg] -> 'sees' | 'likes'
IV[TENSE=pres,  NUM=pl] -> 'disappear' | 'walk'
TV[TENSE=pres, NUM=pl] -> 'see' | 'like'
IV[TENSE=past] -> 'disappeared' | 'walked'
TV[TENSE=past] -> 'saw' | 'liked'


注意一个句法类别可以有多个特征，例如：`V[TENSE=pres, NUM=pl]`。在一般情况下，我们喜欢多少特征就可以添加多少。

In [9]:
# 例9-2. 跟踪基于特征的图表分析器
tokens = 'Kim likes children'.split()
from nltk import load_parser
cp = load_parser('grammars/book_grammars/feat0.fcfg', trace=2)
trees = cp.parse(tokens)

|.Kim .like.chil.|
Leaf Init Rule:
|[----]    .    .| [0:1] 'Kim'
|.    [----]    .| [1:2] 'likes'
|.    .    [----]| [2:3] 'children'
Feature Bottom Up Predict Combine Rule:
|[----]    .    .| [0:1] PropN[NUM='sg'] -> 'Kim' *
Feature Bottom Up Predict Combine Rule:
|[----]    .    .| [0:1] NP[NUM='sg'] -> PropN[NUM='sg'] *
Feature Bottom Up Predict Combine Rule:
|[---->    .    .| [0:1] S[] -> NP[NUM=?n] * VP[NUM=?n] {?n: 'sg'}
Feature Bottom Up Predict Combine Rule:
|.    [----]    .| [1:2] TV[NUM='sg', TENSE='pres'] -> 'likes' *
Feature Bottom Up Predict Combine Rule:
|.    [---->    .| [1:2] VP[NUM=?n, TENSE=?t] -> TV[NUM=?n, TENSE=?t] * NP[] {?n: 'sg', ?t: 'pres'}
Feature Bottom Up Predict Combine Rule:
|.    .    [----]| [2:3] N[NUM='pl'] -> 'children' *
Feature Bottom Up Predict Combine Rule:
|.    .    [----]| [2:3] NP[NUM='pl'] -> N[NUM='pl'] *
Feature Bottom Up Predict Combine Rule:
|.    .    [---->| [2:3] S[] -> NP[NUM=?n] * VP[NUM=?n] {?n: 'pl'}
Feature Single Edge Fundame

In [10]:
# 最后，我们可以检查生成的分析树
for tree in trees: print(tree)

(S[]
  (NP[NUM='sg'] (PropN[NUM='sg'] Kim))
  (VP[NUM='sg', TENSE='pres']
    (TV[NUM='sg', TENSE='pres'] likes)
    (NP[NUM='pl'] (N[NUM='pl'] children))))


#### 术语

到目前为止，我们只看到像 `sg`和`pl` 这样的特征值。这些简单的值通常被称为**原子**。原子值的一种特殊情况是布尔值，例如：我们可能要用布尔特征`AUX`区分助动词。


### 8.2 处理特征结构

在本节中，我们将展示如何构建特征结构，并在 NLTK 中操作。我们还将讨论统一的基本操作，这使我们能够结合两个不同的特征结构中的信息。

In [11]:
fs1 = nltk.FeatStruct(TENSE='past', NUM='sg')
print(fs1)

[ NUM   = 'sg'   ]
[ TENSE = 'past' ]


一个特征结构实际上只是一种字典，所以我们可以平常的方式通过索引访问它的值。我们可以用我们熟悉的方式指定值给特征：

In [12]:
fs1 = nltk.FeatStruct(PER=3, NUM='pl', GND='fem')
print(fs1['GND'])

fem


In [13]:
fs1['CASE'] = 'acc'

In [14]:
fs2 = nltk.FeatStruct(POS='N', AGR=fs1)
print(fs2)

[       [ CASE = 'acc' ] ]
[ AGR = [ GND  = 'fem' ] ]
[       [ NUM  = 'pl'  ] ]
[       [ PER  = 3     ] ]
[                        ]
[ POS = 'N'              ]


In [15]:
print(fs2['AGR'])

[ CASE = 'acc' ]
[ GND  = 'fem' ]
[ NUM  = 'pl'  ]
[ PER  = 3     ]


In [16]:
print(fs2['AGR']['PER'])

3


In [17]:
print(nltk.FeatStruct("[POS='N', AGR=[PER=3, NUM='pl', GND='fem']]"))

[       [ GND = 'fem' ] ]
[ AGR = [ NUM = 'pl'  ] ]
[       [ PER = 3     ] ]
[                       ]
[ POS = 'N'             ]


In [18]:
print(nltk.FeatStruct(name='Lee', telno='01 27 86 42 96', age=33))

[ age   = 33               ]
[ name  = 'Lee'            ]
[ telno = '01 27 86 42 96' ]


将特征结构作为图来查看往往是有用的，更具体的，作为有向无环图（`directed acyclic graphs，DAGs`）

In [19]:
print(nltk.FeatStruct("""[NAME='Lee', ADDRESS=(1)[NUMBER=74, STREET='rue Pascal'], 
SPOUSE=[NAME='Kim', ADDRESS->(1)]]"""))

[ ADDRESS = (1) [ NUMBER = 74           ] ]
[               [ STREET = 'rue Pascal' ] ]
[                                         ]
[ NAME    = 'Lee'                         ]
[                                         ]
[ SPOUSE  = [ ADDRESS -> (1)  ]           ]
[           [ NAME    = 'Kim' ]           ]


#### 包含和统一
认为特征结构提供一些对象的部分信息是很正常的，在这个意义上，我们可以根据它们通用的程度给特征结构排序。这个顺序被称为包含（`subsumption`）；一个更一般的特征结构包含（ `subsumes`）一个较少一般的。

合并两个特征结构的信息被称为统一，由方法 `unify()`支持。

In [20]:
fs1 = nltk.FeatStruct(NUMBER=74, STREET='rue Pascal')
fs2 = nltk.FeatStruct(CITY='Paris')
print(fs1.unify(fs2))

[ CITY   = 'Paris'      ]
[ NUMBER = 74           ]
[ STREET = 'rue Pascal' ]


In [21]:
print(fs2.unify(fs1))

[ CITY   = 'Paris'      ]
[ NUMBER = 74           ]
[ STREET = 'rue Pascal' ]


### 9.3 扩展基于特征的文法

#### 子类别
#### 核心词回顾
#### 助动词与倒装
#### 无限制依赖成分
#### 德语中的格和性别

与英语相比，德语的协议具有相对丰富的形态。例如：在德语中定冠词根据格、性别和 数量变化。

In [22]:
# 例9-3. 具有倒装从句和长距离依赖的产生式的文法，使用斜线类别
nltk.data.show_cfg('grammars/book_grammars/feat1.fcfg')

% start S
# ###################
# Grammar Productions
# ###################
S[-INV] -> NP VP
S[-INV]/?x -> NP VP/?x
S[-INV] -> NP S/NP
S[-INV] -> Adv[+NEG] S[+INV]
S[+INV] -> V[+AUX] NP VP
S[+INV]/?x -> V[+AUX] NP VP/?x
SBar -> Comp S[-INV]
SBar/?x -> Comp S[-INV]/?x
VP -> V[SUBCAT=intrans, -AUX]
VP -> V[SUBCAT=trans, -AUX] NP
VP/?x -> V[SUBCAT=trans, -AUX] NP/?x
VP -> V[SUBCAT=clause, -AUX] SBar
VP/?x -> V[SUBCAT=clause, -AUX] SBar/?x
VP -> V[+AUX] VP
VP/?x -> V[+AUX] VP/?x
# ###################
# Lexical Productions
# ###################
V[SUBCAT=intrans, -AUX] -> 'walk' | 'sing'
V[SUBCAT=trans, -AUX] -> 'see' | 'like'
V[SUBCAT=clause, -AUX] -> 'say' | 'claim'
V[+AUX] -> 'do' | 'can'
NP[-WH] -> 'you' | 'cats'
NP[+WH] -> 'who'
Adv[+NEG] -> 'rarely' | 'never'
NP/NP ->
Comp -> 'that'


In [24]:
tokens = 'who do you claim that you like'.split()
from nltk import load_parser
cp = load_parser('grammars/book_grammars/feat1.fcfg')
for tree in cp.parse(tokens):
    print(tree)

(S[-INV]
  (NP[+WH] who)
  (S[+INV]/NP[]
    (V[+AUX] do)
    (NP[-WH] you)
    (VP[]/NP[]
      (V[-AUX, SUBCAT='clause'] claim)
      (SBar[]/NP[]
        (Comp[] that)
        (S[-INV]/NP[]
          (NP[-WH] you)
          (VP[]/NP[] (V[-AUX, SUBCAT='trans'] like) (NP[]/NP[] )))))))


In [25]:
# 例9-4. 基于特征的文法的例子
nltk.data.show_cfg('grammars/book_grammars/german.fcfg')

% start S
# Grammar Productions
S -> NP[CASE=nom, AGR=?a] VP[AGR=?a]
NP[CASE=?c, AGR=?a] -> PRO[CASE=?c, AGR=?a]
NP[CASE=?c, AGR=?a] -> Det[CASE=?c, AGR=?a] N[CASE=?c, AGR=?a]
VP[AGR=?a] -> IV[AGR=?a]
VP[AGR=?a] -> TV[OBJCASE=?c, AGR=?a] NP[CASE=?c]
# Lexical Productions
# Singular determiners
# masc
Det[CASE=nom, AGR=[GND=masc,PER=3,NUM=sg]] -> 'der' 
Det[CASE=dat, AGR=[GND=masc,PER=3,NUM=sg]] -> 'dem'
Det[CASE=acc, AGR=[GND=masc,PER=3,NUM=sg]] -> 'den'
# fem
Det[CASE=nom, AGR=[GND=fem,PER=3,NUM=sg]] -> 'die' 
Det[CASE=dat, AGR=[GND=fem,PER=3,NUM=sg]] -> 'der'
Det[CASE=acc, AGR=[GND=fem,PER=3,NUM=sg]] -> 'die' 
# Plural determiners
Det[CASE=nom, AGR=[PER=3,NUM=pl]] -> 'die' 
Det[CASE=dat, AGR=[PER=3,NUM=pl]] -> 'den' 
Det[CASE=acc, AGR=[PER=3,NUM=pl]] -> 'die' 
# Nouns
N[AGR=[GND=masc,PER=3,NUM=sg]] -> 'Hund'
N[CASE=nom, AGR=[GND=masc,PER=3,NUM=pl]] -> 'Hunde'
N[CASE=dat, AGR=[GND=masc,PER=3,NUM=pl]] -> 'Hunden'
N[CASE=acc, AGR=[GND=masc,PER=3,NUM=pl]] -> 'Hunde'
N[AGR=[GND=fem,PER=3,

In [27]:
tokens = 'ich folge den Katzen'.split()
cp = load_parser('grammars/book_grammars/german.fcfg')
for tree in cp.parse(tokens):
    print(tree)

(S[]
  (NP[AGR=[NUM='sg', PER=1], CASE='nom']
    (PRO[AGR=[NUM='sg', PER=1], CASE='nom'] ich))
  (VP[AGR=[NUM='sg', PER=1]]
    (TV[AGR=[NUM='sg', PER=1], OBJCASE='dat'] folge)
    (NP[AGR=[GND='fem', NUM='pl', PER=3], CASE='dat']
      (Det[AGR=[NUM='pl', PER=3], CASE='dat'] den)
      (N[AGR=[GND='fem', NUM='pl', PER=3]] Katzen))))


### 9.4 小结

- 上下文无关文法的传统分类是原子符号。特征结构的一个重要的作用是捕捉精细的区分，否则将需要数量翻倍的原子类别。
- 通过使用特征值上的变量，我们可以表达文法产生式中的限制，允许不同的特征规格的实现可以相互依赖。
- 通常情况下，我们在词汇层面指定固定的特征值，限制短语中的特征值与它们的孩子中的对应值统一。
- 特征值可以是原子的或复杂的。原子值的一个特定类别是布尔值，按照惯例用`[+/- feat]`表示。
- 两个特征可以共享一个值（原子的或复杂的）。具有共享值的结构被称为重入。共享的值被表示为`AVM` 中的数字索引（或标记）。
- 一个特征结构中的路径是一个特征的元组，对应从图的根开始的弧的序列上的标签。
- 两条路径是等价的，如果它们共享一个值。
- 包含的特征结构是偏序的。`FS0`包含 `FS1`，当 `FS0`比`FS1`更一般（较少信息）。
- 两种结构`FS0` 和`FS1`的统一，如果成功，就是包含 `FS0`和`FS1`的合并信息的特征结构 `FS2`。
- 如果统一在`FS` 中指定一条路径`π`，那么它也指定等效与`π`的每个路径`π'`。
- 我们可以使用特征结构建立对大量广泛语言学现象的简洁的分析，包括动词子类别，倒装结构，无限制依赖结构和格支配。